# Import Required Libraries

In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras_preprocessing import sequence
from keras.utils import to_categorical
from keras.models import load_model

# Reading Dataset

In [11]:
df = pd.read_csv("/content/spam.csv", encoding="latin-1")

In [13]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


# Preprocessing

In [14]:
#dropping unwanted columns
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [15]:
df.groupby(['v1']).size()

v1
ham     4825
spam     747
dtype: int64

In [17]:
X = df.v2
Y = df.v1

In [18]:
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [19]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [20]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

# Creating LSTM Model

In [22]:
inputs = Input(name='InputLayer',shape=[max_len])

# Adding Layers

In [24]:
layer = Embedding(max_words,50,input_length=max_len)(inputs)
layer = LSTM(64)(layer)
layer = Dense(256,name='FullyConnectedLayer1')(layer)
layer = Activation('relu')(layer)
layer = Dropout(0.5)(layer)
layer = Dense(1,name='OutputLayer')(layer)
layer = Activation('sigmoid')(layer)
model = Model(inputs=inputs,outputs=layer)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 InputLayer (InputLayer)     [(None, 150)]             0         
                                                                 
 embedding_2 (Embedding)     (None, 150, 50)           50000     
                                                                 
 lstm_2 (LSTM)               (None, 64)                29440     
                                                                 
 FullyConnectedLayer1 (Dense  (None, 256)              16640     
 )                                                               
                                                                 
 activation_4 (Activation)   (None, 256)               0         
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                             

# Compile the Model

In [25]:
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

# Fit the Model

In [26]:
model.fit(
    sequences_matrix,
    Y_train,
    batch_size=128,
    epochs=10,
    validation_split=0.2
)

Epoch 1/10
30/30 [==============================] - 8s 30ms/step - loss: 0.3239 - accuracy: 0.8794 - val_loss: 0.1379 - val_accuracy: 0.9778
Epoch 2/10
30/30 [==============================] - 0s 13ms/step - loss: 0.0942 - accuracy: 0.9768 - val_loss: 0.0527 - val_accuracy: 0.9884
Epoch 3/10
30/30 [==============================] - 0s 13ms/step - loss: 0.0433 - accuracy: 0.9868 - val_loss: 0.0499 - val_accuracy: 0.9884
Epoch 4/10
30/30 [==============================] - 0s 14ms/step - loss: 0.0336 - accuracy: 0.9902 - val_loss: 0.0532 - val_accuracy: 0.9863
Epoch 5/10
30/30 [==============================] - 0s 14ms/step - loss: 0.0257 - accuracy: 0.9942 - val_loss: 0.0530 - val_accuracy: 0.9895
Epoch 6/10
30/30 [==============================] - 0s 13ms/step - loss: 0.0188 - accuracy: 0.9947 - val_loss: 0.0598 - val_accuracy: 0.9884
Epoch 7/10
30/30 [==============================] - 0s 14ms/step - loss: 0.0155 - accuracy: 0.9958 - val_loss: 0.0632 - val_accuracy: 0.9895
Epoch 8/10
30

# Save the Model

In [28]:
model.save("SMS_Spam_Classification")

# Test the Model

In [32]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix  = sequence.pad_sequences(test_sequences,maxlen=max_len)\

# Evaluating the model
accuracy = model.evaluate(test_sequences_matrix,Y_test)
print(f"Accuracy: {accuracy[1]:0.2f}")

27/27 [==============================] - 0s 6ms/step - loss: 0.1160 - accuracy: 0.9761
Accuracy: 0.98


## Predict the model

In [34]:
y_pred = model.predict(test_sequences_matrix)

27/27 [==============================] - 0s 5ms/step


In [35]:
y_pred[25:40].round(3)

array([[0.003],
       [0.   ],
       [0.006],
       [0.001],
       [1.   ],
       [0.   ],
       [0.   ],
       [0.   ],
       [0.001],
       [0.   ],
       [0.   ],
       [1.   ],
       [0.   ],
       [0.   ],
       [0.   ]], dtype=float32)

In [38]:
Y_test[25:40]

array([[0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0]])